## TPC-DS Query Evaluation  
Evaluate performance of suite of TPC queries

In [1]:
from google.cloud import bigquery

In [2]:
import config

In [3]:
client = bigquery.Client.from_service_account_json(config.gcp_cred_file)

In [4]:
job_config = bigquery.QueryJobConfig()

In [5]:
job_config.default_dataset = "tpc-benchmarking-9432.ds_2GB_1_basic"

In [13]:
fp = '/home/colin/code/bq_snowflake_benchmark/q/ds_01_1GB_bq_default_qual/query_0.sql'
text = open(fp, 'r').read()
query_job = client.query(text, job_config=job_config)
result = query_job.result()

BadRequest: 400 Syntax error: Expected ")" but got integer literal "14" at [282:69]

(job ID: 568bc12b-2bab-4985-ab47-f9ccdf3a41d8)

                                                    -----Query Job SQL Follows-----                                                    

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:with customer_total_return as
   3:(select sr_customer_sk as ctr_customer_sk
   4:,sr_store_sk as ctr_store_sk
   5:,sum(SR_FEE) as ctr_total_return
   6:from store_returns
   7:,date_dim
   8:where sr_returned_date_sk = d_date_sk
   9:and d_year =2000
  10:group by sr_customer_sk
  11:,sr_store_sk)
  12: select  c_customer_id
  13:from customer_total_return ctr1
  14:,store
  15:,customer
  16:where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
  17:from customer_total_return ctr2
  18:where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
  19:and s_store_sk = ctr1.ctr_sto

### 